In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import numpy  as np
import seaborn as sns
%matplotlib inline
import plotly.io as pio
pio.renderers.default = "notebook"
path = './data/'
pd.options.display.max_rows = 150
pd.options.display.max_columns = 350
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

# 데이터로드
<pre>
sido : 15년
sido2: 15-17년
</pre>

### sido - 지표로드

In [33]:
sido = pd.read_csv(path+'시도별지역안전지표_2015.csv', encoding='cp949')
sido_w = pd.read_csv(path+'sido_wide.csv', encoding='cp949')
sido.dropna(axis=1, inplace=True)
sido['범죄bin']=np.zeros(17)
sido.loc[(sido.범죄==4)|(sido.범죄==5),'범죄bin']=1
sido.loc[~((sido.범죄==4)|(sido.범죄==5)),'범죄bin']=0

### sido - 지표추가

In [34]:
이혼통계2='C:/Users/COM/Desktop/이혼통계2/'
미성년자녀수별 = pd.read_csv(이혼통계2+'시도_미성년자녀수별_이혼.csv', encoding='cp949')
천명당소년범죄 = pd.read_csv(이혼통계2+'시도_소년천명당_소년발생건수.csv', encoding='cp949')
이혼사유별 = pd.read_csv(이혼통계2+'시도_이혼사유별_이혼.csv', encoding='cp949')
이혼종류별 = pd.read_csv(이혼통계2+'시도_이혼종류별_이혼.csv', encoding='cp949')

In [35]:
# 미성년자녀가 있는 이혼이 많은 지역은 소년범죄 (혹은 범죄)가 많을 것이다.
미성년자녀수별.rename(columns={'시도별':'시도'},inplace=True)
# (배우자부정, 가족간불화) 정신적육체적학대로 인한 이혼이 많은 지역은 소년범죄가 많을 것이다.
이혼사유별.rename(columns={'시도별':'시도','계':'이혼사유_계','기타':'이혼사유_기타'},inplace=True)
# 재판이혼이 많은 지역은 소년범죄가 많을 것이다.
이혼종류별.rename(columns={'시도별':'시도','계':'이혼종류_계'},inplace=True)
# 소년범죄 비율이 높을 것이다.
천명당소년범죄.rename(columns={'행정구역별':'시도','2015':'천명당소년범죄'},inplace=True)

In [36]:
미성년자녀수별['미성년자녀유이혼건수']=미성년자녀수별['1명']+미성년자녀수별['2명']+미성년자녀수별['3명이상']
미성년자녀수별['미성년자녀유이혼비율']=미성년자녀수별['미성년자녀유이혼건수']/미성년자녀수별['이혼건수-계']
이혼사유별['확인건수']=이혼사유별['배우자 부정']+이혼사유별['정신적육체적 학대']+이혼사유별['가족간 불화']
이혼사유별['확인비율']=이혼사유별['확인건수']/이혼사유별['이혼사유_계']
이혼사유별['학대비율']=이혼사유별['정신적육체적 학대']/이혼사유별['이혼사유_계']
이혼종류별['재판이혼비율']=이혼종류별['재판이혼']/이혼종류별['이혼종류_계']

In [37]:
시도추가정보 = 미성년자녀수별.merge(이혼사유별).merge(이혼종류별).merge(천명당소년범죄)

In [38]:
sido=sido.merge(시도추가정보)
# sido.head(1)

### sido2 - 지표추가

In [39]:
미성년자녀수별2 = pd.read_csv(이혼통계2+'시도_미성년자녀수별_이혼_1318.csv', encoding='cp949')
미성년자녀수별2['미성년자녀-있음']=미성년자녀수별2['1명']+미성년자녀수별2['2명']+미성년자녀수별2['3명이상']
미성년자녀수별2['미성년자녀-있음비율']=미성년자녀수별2['미성년자녀-있음']/미성년자녀수별2['이혼건수-계']

In [40]:
범죄등급1518 = pd.read_csv(이혼통계2+'16-18시도범죄등급.csv', encoding='cp949')

In [41]:
천명당소년범죄2 = pd.read_csv(이혼통계2+'시도_소년천명당_소년발생건수_1317.csv', encoding='cp949')
천명당소년범죄2.rename(columns={'행정구역별':'시도별'},inplace=True)

In [167]:
이혼사유별2 = pd.read_csv(이혼통계2+'시도_이혼사유별_이혼_1317.csv', encoding='cp949')
이혼사유별2['정신적육체적 학대비율']=이혼사유별2['정신적육체적 학대']/이혼사유별2['계']
이혼사유별2['부정적가정상황 건수']=이혼사유별2['정신적육체적 학대']+이혼사유별2['가족간 불화']+이혼사유별2['배우자 부정']
이혼사유별2['부정적가정상황 비율']=이혼사유별2['부정적가정상황 건수']/이혼사유별2['계']
이혼사유별2['기타상황 비율']=(이혼사유별2['기타'])/이혼사유별2['계']
# 이혼사유별2['경제문제']
# 경제문제ㅡ성격차이, 건강문제, 배우자부정, 가족간불화
이혼사유별2

,시도별,년도,계,배우자 부정,정신적육체적 학대,가족간 불화,경제문제,성격차이,건강문제,기타,미상,정신적육체적 학대비율,부정적가정상황 건수,부정적가정상황 비율,기타상황 비율
0,전국,13,115292,8702,4784,7961,14519,53894,745,23494,1193,0.041495,21447,0.186023,0.203778
1,서울특별시,13,20126,1416,697,1121,2551,9635,110,4352,244,0.034632,3234,0.160688,0.216238
2,부산광역시,13,7444,496,321,482,976,3624,43,1471,31,0.043122,1299,0.174503,0.197609
3,대구광역시,13,4796,325,187,576,751,2451,73,415,18,0.038991,1088,0.226856,0.086530
4,인천광역시,13,7253,558,310,515,1125,3337,36,1269,103,0.042741,1383,0.190680,0.174962
5,광주광역시,13,2924,211,111,173,316,1305,14,785,9,0.037962,495,0.169289,0.268468
6,대전광역시,13,3003,196,100,183,359,1646,14,495,10,0.033300,479,0.159507,0.164835
7,울산광역시,13,2578,211,129,272,346,1169,12,425,14,0.050039,612,0.237393,0.164856
8,세종특별자치시,13,240,13,10,26,20,121,0,50,0,0.041667,49,0.204167,0.208333
9,경기도,13,28733,2258,1165,1819,3802,13067,158,6034,430,0.040546,5242,0.182438,0.210002


In [168]:
이혼종류별2 = pd.read_csv(이혼통계2+'시도_이혼종류별_이혼_1318.csv', encoding='cp949')
이혼종류별2['재판이혼 비율']=이혼종류별2['재판이혼']/이혼종류별2['계']

In [169]:
sido2 =미성년자녀수별2.merge(이혼사유별2,on=['시도별','년도']).merge(이혼종류별2,on=['시도별','년도']).merge(천명당소년범죄2,on=['시도별','년도']).merge(범죄등급1518,on=['시도별','년도'])
# 여기는 추가정보만으로 해야됨
# 시도추가정보2.head(1)

In [170]:
#범죄등급 binary 추가
sido2.loc[(sido2.범죄==4)|(sido2.범죄==5),'범죄bin']=int(1)
sido2.loc[~((sido2.범죄==4)|(sido2.범죄==5)),'범죄bin']=int(0)

# 함수선언
### 중앙선그리는 함수

In [171]:
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
def scatter(x, y, color, df, hovername=None,text=False, medianline=False, palette=['mediumorchid','dimgrey'], trendline=None):
    #산점도------
    if text == True:
        fig = px.scatter(df, x=x, y=y, text='시군구',trendline=trendline,
                             hover_name=hovername, color=color, color_discrete_sequence=palette)
        fig.update_traces(textposition='top center')
    else:
        fig = px.scatter(df, x=x, y=y,trendline=trendline,
                         hover_name=hovername, color=color, color_discrete_sequence=palette)
        fig.update_traces(textposition='top center')    
    #중앙값-----
    if medianline==True:
        #x median
        fig.add_trace(go.Scatter(x=[df[x].median()]*2,
                       y=[-10000000,10000000],
                       name="median x: "+str(round(df[x].median(),2)),
                       mode="lines",
                       line = dict(color = ('rgb(0,0,0)'),width=0.5),
                       visible=True,))
        #y median
        fig.add_trace(go.Scatter(x=[-10000000,10000000],
                       y=[df[y].median()]*2,
                       name="median y: "+str(round(df[y].median(),2)),
                       mode="lines",
                       line = dict(color = ('rgb(0,0,0)'),width=0.5),
                       visible=True))
        # 레전이동      fig.update_layout(legend=dict(x=-.05, y=1.15))
    
    #트렌드라인----
    if trendline==True:
        regline = sm.OLS(df[y],sm.add_constant(df[x])).fit().fittedvalues
        fig.add_traces(go.Scatter(x=df[x], y=regline,
                              mode = 'lines',
                              marker_color='black',
                            line = dict(color = ('rgb(0,0,0)'),width=0.5,dash='7px'),
                              name='trend all'))  
    #플롯 레이아웃(xy범위)----
    fig.update_xaxes(range=[df[x].min()*0.8, df[x].max()*1.05]) #min값 음수인지 아닌지 확인 후 범위 조절
    fig.update_yaxes(range=[df[y].min()*0.5, df[y].max()*1.05]) #min값 음수인지 아닌지 확인 후 범위 조절
    fig.show()

### 회귀식기준 위아래 갯수세기 함수

In [172]:
from scipy import stats
def total_regression(x_,y_, totaldf,groupdf):
    #전체에 대한 xy
    x = totaldf[x_].tolist()
    y = totaldf[y_].tolist()
    #전체에 대한 회귀식생성
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    #지표출력
    print(stats.linregress(x,y))
    print('Rsquared=',stats.linregress(x,y)[2]**2)
        #group에 포함되는 xy
    x = groupdf[x_].tolist()
    y = groupdf[y_].tolist()
    y_pred = []
    for i in range(len(x)):
        y_pred.append(slope*np.array(x[i])+intercept)
    upper = 0
    lower = 0
    for i in range(len(y)):
        if y[i]>=y_pred[i]:
            upper +=1
        else:
            lower +=1
    return pd.DataFrame({'':['total','upper','lower'],
                          'count':[len(y),upper,lower],
                          'ratio':[len(y)/len(y), upper/len(y), lower/len(y)]})

### 사분면별 점갯수세기

In [183]:
# 사분면별 점 갯수 세는 함수
def 사분면별점갯수(x,y,스트링등급,df):
    condlist=[1,(df[스트링등급]=='1'),(df[스트링등급]=='0')]
    for i,cond in enumerate(condlist):
        quad1 = len(df.loc[(df[x]>=df[x].median())&(df[y]>=df[y].median())&cond])
        quad2 = len(df.loc[(df[x]<=df[x].median())&(df[y]>=df[y].median())&cond])
        quad3 = len(df.loc[(df[x]<=df[x].median())&(df[y]<=df[y].median())&cond])
        quad4 = len(df.loc[(df[x]>=df[x].median())&(df[y]<=df[y].median())&cond])
        if i==0:
            total = [quad1,quad2,quad3,quad4]
            tot_12 = quad1+quad2
            tot_34 = quad3+quad4
            tot_14 = quad1+quad4
            tot_23 = quad2+quad3
        elif i==1:
            num1 = [quad1,quad2,quad3,quad4]
            num1_12 = quad1+quad2
            num1_34 = quad3+quad4
            num1_14 = quad1+quad4
            num1_23 = quad2+quad3
        elif i==2:
            num2 = [quad1,quad2,quad3,quad4]
            num2_12 = quad1+quad2
            num2_34 = quad3+quad4
            num2_14 = quad1+quad4
            num2_23 = quad2+quad3
    return pd.DataFrame({'quad':['quad1','quad2','quad3','quad4','1&2','3&4','1&4','2&3'],\
                         '1(색)':num1+[num1_12,num1_34,num1_14,num1_23],\
                         '0(흑)':num2+[num2_12,num2_34,num2_14,num2_23],\
                         'total':total+[tot_12,tot_34,tot_14,tot_23]})

# 그룹나누기

In [184]:
# #1사분면의 모든 주황초록점 : group1, 1사분면 주황점: group2
# sigungu_nxy['group1'] = np.zeros(len(sigungu_nxy))
# sigungu_nxy['group2'] = np.zeros(len(sigungu_nxy))
# sigungu_nxy['group3'] = np.zeros(len(sigungu_nxy))
# sigungu_nxy['group4'] = np.zeros(len(sigungu_nxy))
# #조건을 만족하는 행 인덱스 가져오기
# cond = (sigungu_nxy['2세대미혼자녀가구합']>sigungu_nxy['2세대미혼자녀가구합'].median())
# cond2 = (sigungu_nxy['불안정가구비율']>sigungu_nxy['불안정가구비율'].median())
# group1_index = sigungu_nxy.loc[cond].index
# group2_index = sigungu_nxy.loc[cond&sigungu_nxy.crime_==1].index
# group3_index = sigungu_nxy.loc[cond2].index
# group4_index = sigungu_nxy.loc[cond2&sigungu_nxy.crime_==1].index
# #group1에 속하는 애들을 1로 표시
# sigungu_nxy.loc[sigungu_nxy.index.isin(group1_index),'group1'] = 1
# sigungu_nxy.loc[sigungu_nxy.index.isin(group2_index),'group2'] = 1
# sigungu_nxy.loc[sigungu_nxy.index.isin(group3_index),'group3'] = 1
# sigungu_nxy.loc[sigungu_nxy.index.isin(group4_index),'group4'] = 1
# sigungu_nxy['group1_'] = sigungu_nxy.group1.astype(str)
# sigungu_nxy['group2_'] = sigungu_nxy.group2.astype(str)
# sigungu_nxy['group3_'] = sigungu_nxy.group3.astype(str)
# sigungu_nxy['group4_'] = sigungu_nxy.group4.astype(str)
# df.columns=['시도','시군구','소년범죄건수','범죄건수','소년폭력범죄건수','폭력범죄건수','소년범죄비율','소년폭력범죄비율']

# scatter plot

## 가설: 이혼부모 + 미성년자녀 가구가 많은 지역은 소년범죄율이 높을 것이다. - 기각
## 재판이혼이 많은 지역은 범죄가 많을 것이다.
## 재판이혼이 많은 지역은 소년범죄가 많을 것이다

In [185]:
#그룹분류를 위한 스트링등급 추가
sido2['범죄bin_']=sido2.범죄bin.astype(int).astype(str)
sido2['지역'] = sido2['시도별'].astype(str) + '_' + sido2['년도'].astype(str)
sido2.head(1)

,시도별,년도,이혼건수-계,미성년자녀-없음,1명,2명,3명이상,미상_x,미성년자녀-있음,미성년자녀-있음비율,계_x,배우자 부정,정신적육체적 학대,가족간 불화,경제문제,성격차이,건강문제,기타,미상_y,정신적육체적 학대비율,부정적가정상황 건수,부정적가정상황 비율,기타상황 비율,계_y,협의이혼,재판이혼,재판이혼 비율,천명당소년범죄건수,범죄,범죄bin,범죄bin_,지역
0,서울특별시,15,18176,9904,4639,2935,391,307,7965,0.438215,18176,1142,600,1362,2018,8309,84,3843,818,0.033011,3104,0.170775,0.211433,18176,13981,4194,0.230744,13.94,4,1.0,1,서울특별시_15


<pre>*변수
미성년자녀-있음, 미성년자녀-있음비율
정신적육체적 학대, 정신적육체적 학대비율
부정적가정상황 건수, 부정적가정상황 비율
재판이혼, 재판이혼 비율
천명당소년범죄건수
범죄,범죄bin,범죄bin_
</pre>

<pre>*찾을것
그래프 그렸을 때 범죄등급대로 쏠리는지?
그룹해서 소년범죄율을 봤을 때 그룹대로 쏠리는지</pre>

In [186]:
scatter(x="기타상황 비율", y="정신적육체적 학대비율", color="범죄bin_", df=sido2,
        medianline=True, hovername='지역',trendline='ols')
사분면별점갯수('기타상황 비율','정신적육체적 학대비율','범죄bin_',sido2)

,quad,1(색),0(흑),total
0,quad1,3,9,12
1,quad2,2,10,12
2,quad3,8,4,12
3,quad4,5,7,12
4,1&2,5,19,24
5,3&4,13,11,24
6,1&4,8,16,24
7,2&3,10,14,24


In [188]:
scatter(x="미성년자녀-있음비율", y="정신적육체적 학대비율", color="범죄bin_", df=sido2,
        medianline=True, hovername='지역',trendline='ols')
사분면별점갯수('미성년자녀-있음비율','정신적육체적 학대비율','범죄bin_',sido2)
#이혼사유: 정신육체적학대비율이 낮을수록 범죄등급이 높은 경향 있음 6:13 --> 왜?
#미성년자녀: 있음비율은 범죄등급에 영향을 미치지 않음 11:8

,quad,1(색),0(흑),total
0,quad1,2,14,16
1,quad2,3,5,8
2,quad3,9,7,16
3,quad4,4,4,8
4,1&2,5,19,24
5,3&4,13,11,24
6,1&4,6,18,24
7,2&3,12,12,24


In [189]:
scatter(x="부정적가정상황 비율", y="재판이혼 비율", color="범죄bin_", df=sido2,
        medianline=True, hovername='지역',trendline='ols')
사분면별점갯수('부정적가정상황 비율','재판이혼 비율','범죄bin_',sido2)
#이혼사유: 부정정가정상황이 낮을수록 범죄등급이 높은 경향 있음 12:6 (기각) --> 왜? 나머지사유로 해볼까
#이혼종류: 재판이혼비율은 범죄등급에 영향을 미치지 않음 10:8

,quad,1(색),0(흑),total
0,quad1,3,9,12
1,quad2,7,5,12
2,quad3,5,7,12
3,quad4,3,9,12
4,1&2,10,14,24
5,3&4,8,16,24
6,1&4,6,18,24
7,2&3,12,12,24


In [190]:
xlist = ['재판이혼 비율','정신적육체적 학대비율','부정적가정상황 비율','미성년자녀-있음비율']
for x in xlist:
    scatter(x=x, y="천명당소년범죄건수", color="범죄bin_", df=sido2,
            medianline=True, hovername='지역',trendline='ols')
    print(사분면별점갯수(x,'천명당소년범죄건수','범죄bin_',sido2))

    quad  1(색)  0(흑)  total
0  quad1     5     5     10
1  quad2     5     9     14
2  quad3     3     7     10
3  quad4     5     9     14
4    1&2    10    14     24
5    3&4     8    16     24
6    1&4    10    14     24
7    2&3     8    16     24


    quad  1(색)  0(흑)  total
0  quad1     3     9     12
1  quad2     7     5     12
2  quad3     6     6     12
3  quad4     2    10     12
4    1&2    10    14     24
5    3&4     8    16     24
6    1&4     5    19     24
7    2&3    13    11     24


    quad  1(색)  0(흑)  total
0  quad1     3    10     13
1  quad2     7     4     11
2  quad3     5     8     13
3  quad4     3     8     11
4    1&2    10    14     24
5    3&4     8    16     24
6    1&4     6    18     24
7    2&3    12    12     24


    quad  1(색)  0(흑)  total
0  quad1     3     9     12
1  quad2     7     5     12
2  quad3     5     7     12
3  quad4     3     9     12
4    1&2    10    14     24
5    3&4     8    16     24
6    1&4     6    18     24
7    2&3    12    12     24


In [26]:
# def 그룹핑(기준1,기준2,df):
#     df['group'] = np.zeros(len(df))
#     cond = (df[기준1]>df[기준1].median())
#     cond2 = (df[기준2]>df[기준2].median())
#     group_index = df.loc[cond&cond2].index
#     df.loc[df.index.isin(group_index),'group'] = 1
#     df['group_'] = df.group.astype(int).astype(str)

In [28]:
# 그룹핑('정신적육체적 학대비율','미성년자녀-있음비율',sido2)
# sido2.head(5)

,시도별,년도,이혼건수-계,미성년자녀-없음,1명,2명,3명이상,미상_x,미성년자녀-있음,미성년자녀-있음비율,계_x,배우자 부정,정신적육체적 학대,가족간 불화,경제문제,성격차이,건강문제,기타,미상_y,정신적육체적 학대비율,부정적가정상황 건수,부정적가정상황 비율,계_y,협의이혼,재판이혼,재판이혼 비율,천명당소년범죄건수,범죄,범죄bin,범죄bin_,지역,group,group_
0,서울특별시,15,18176,9904,4639,2935,391,307,7965,0.438215,18176,1142,600,1362,2018,8309,84,3843,818,0.033011,3104,0.170775,18176,13981,4194,0.230744,13.94,4,1.0,1,서울특별시_15,0.0,0
1,부산광역시,15,6649,3558,1640,1237,157,57,3034,0.456309,6649,438,231,445,787,3515,45,1075,113,0.034742,1114,0.167544,6649,5230,1419,0.213416,21.61,4,1.0,1,부산광역시_15,0.0,0
2,대구광역시,15,4497,2199,1221,943,129,5,2293,0.509895,4497,396,203,341,488,2010,33,975,51,0.045141,940,0.209028,4497,3431,1066,0.237047,14.00,3,0.0,0,대구광역시_15,1.0,1
3,인천광역시,15,7116,3595,1868,1405,229,19,3502,0.492130,7116,596,286,463,880,3419,40,1257,175,0.040191,1345,0.189011,7116,5651,1465,0.205874,15.28,2,0.0,0,인천광역시_15,0.0,0
4,광주광역시,15,2842,1278,762,666,131,5,1559,0.548557,2842,220,134,234,450,1157,21,549,77,0.047150,588,0.206897,2842,2149,693,0.243842,14.66,5,1.0,1,광주광역시_15,1.0,1


In [35]:
# 그룹핑셀
기준1 = '재판이혼 비율'
기준2 = '미성년자녀-있음비율'
sido2['group'] = np.zeros(len(sido2))
cond = (sido2[기준1]<sido2[기준1].median())
cond2 = (sido2[기준2]<sido2[기준2].median())
group_index = sido2.loc[cond&cond2].index
sido2.loc[sido2.index.isin(group_index),'group'] = 1
sido2['group_'] = sido2.group.astype(int).astype(str)

In [36]:
#팔레트정의
darkcyan= ['dimgrey','darkcyan']
darkorange= ['darkorange','dimgrey']

In [38]:
scatter(x="정신적육체적 학대비율", y="천명당소년범죄건수", color="group_", df=sido2,
        medianline=True, hovername='지역',palette=darkcyan,trendline='ols')

In [69]:
sido['group1'] = np.zeros(len(sido))
cond = (sido['미성년자녀유이혼비율']>sido['미성년자녀유이혼비율'].median())
cond2 = (sido['학대비율']>sido['학대비율'].median())
group1_index = sido.loc[cond&cond2].index
sido.loc[sido.index.isin(group1_index),'group1'] = 1
sido['group1_'] = sido.group1.astype(str)

In [71]:
scatter(x="천명당소년범죄", y="소년범죄_건수총합", color="crime_", df=sigungu_nxy,
        medianline=True, hovername='지역',trendline='ols')
scatter(x="불안정가구비율", y="소년범죄_전체비율", color="crime_", df=sigungu_nxy,
        medianline=True, hovername='지역',trendline='ols')
scatter(x="학교수(고등학교)", y="소년범죄_건수총합", color="crime_", df=sigungu_nxy,
        medianline=True, hovername='지역',trendline='ols')

NameError: name 'sigungu_nxy' is not defined

In [ ]:
scatter(x="전체범죄_건수총합", y="소년범죄_건수총합", color="group3_", df=sigungu_nxy,
        medianline=1, palette = darkcyan, hovername='지역', trendline='ols')
scatter(x="전체범죄_건수총합", y="소년범죄_전체비율", color="group3_", df=sigungu_nxy,
        medianline=1, palette = darkcyan, hovername='지역', trendline='ols')

In [ ]:
scatter(x="5대범죄 발생건수", y="소년범죄_건수총합", color="group1_", df=sigungu_nxy,
        medianline=True, palette = darkcyan,hovername='지역', trendline='ols')
scatter(x="5대범죄 발생건수", y="소년범죄_전체비율", color="group1_", df=sigungu_nxy,
        medianline=True, palette = darkcyan,hovername='지역', trendline='ols')

In [ ]:
# 연속형 변수 중에 DI1_pr에 따라 평균이 다른 변수만 추출
import matplotlib.pyplot as plt
import seaborn as sns # plt, sns는 그림을 그리기 위함
import scipy.stats as stats # t-test를 하기 위함
# label1='제조업비율 높음',label2='제조업비율 낮음'
def con_check(var1, var2, data, label1='높음', label2='낮음'): # var1에는 질병에 관한 변수, var2는 연속형 변수들, data는 확인할 dataframe
    var1_0 = data.loc[data[var1] == 0][var2] # var1==0인 경우의 var2값
    var1_1 = data.loc[data[var1] == 1][var2] # var1==1인 경우의 var2값
    g = sns.distplot(var1_0, label = label1)
    g = sns.distplot(var1_1, label = label2) #각 경우의 분포 그래프 그리기
    g.legend()
    plt.show()
    
    
    # t-test하기 전에 먼저 두 집단의 var2변수가 등분산을 충족하는지 검정
    # H0(귀무가설) : 두 모집단의 분산은 같다
    # H1(대립가설) : 두 모집단의 분산은 다르다
    equal_variance=stats.levene(var1_0, var1_1)
    print('등분산 검정에 대한 p-value : %s'%equal_variance[1]) # 등분산 검정에 대한 p-value
    
    
    # 독립표본 T검정
    #H0(귀무가설) : var1에 따른 var2의 평균에 차이가 없다
    #H1(대립가설) : var1에 따른 var2의 평균에 차이가 있다
     
    
    # 등분산 검정에 대한 p-value가 0.05보다 작은 경우, H0를 기각하므로 두 모집단의 분산은 다른다
    if equal_variance[1] < 0.05:
        print('등분산 검정에 대한 p-value가 0.05보다 작다')
        # 두 모집단의 분산이 다른 경우, equal_var=False로 놓고 독립표본 t-검정 진행
        t = stats.ttest_ind(var1_0, var1_1, equal_var=False)
        print('t-test에 대한 p-vlaue : %s'%t[1])
        if t[1] < 0.05:
            print('T-test에 대한 p-value가 0.05보다 작으므로 HO를 기각한다★')
            print(' a집단 음주 평균:',var1_0.mean(),'\n','b집단 음주 평균:',var1_1.mean())
            print('차이:', abs(var1_0.mean()-var1_1.mean()))
            return var2
        else:
            print('T-test에 대한 p-value가 0.05보다 크므로 HO를 채택')
            
    # 등분산 검정에 대한 p-value가 0.05보다 큰 경우, H0를 채택하므로 두 모집단의 분산은 같다.
    else: 
        # 두 모집단의 분산이 같은 경우, equal_var=True로 놓고 독립표본 t-검정 진행
        t = stats.ttest_ind(var1_0, var1_1, equal_var=True)
        print('t-test에 대한 p-vlaue : %s'%t[1])
        
        if t[1] < 0.05:
            print('T-test에 대한 p-value가 0.05보다 작으므로 HO를 기각한다★')
            print(' a집단 음주 평균:',var1_0.mean(),'\n','b집단 음주 평균:',var1_1.mean())
            print('차이:', abs(var1_0.mean()-var1_1.mean()))
            return var2
        else:
            print('T-test에 대한 p-value가 0.05보다 크므로 HO를 채택')

In [ ]:
con_check('group4','소년범죄_전체비율',sigungu_nxy)